In [ ]:
!python -m pip install --upgrade pip setuptools wheel -q
!python -m pip install lbry-libtorrent icecream -q
!apt install python3-libtorrent -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.5 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libtorrent-rasterbar2.0
The following NEW packages will be installed:
  libtorrent-rasterbar2.0 python3-libtorrent
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,098 kB of archives.
After this operation, 7,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtorrent-rasterbar2.0 amd64 2.0.5-5 [1,501 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-libtorrent am

In [ ]:
import os
import time
import math
import shutil
import threading
from icecream import ic
import libtorrent as lt
import ipywidgets as widgets
from dataclasses import dataclass
from IPython.display import display

In [ ]:
class Category:

  def __init__(
    self,
    name: str,
    directory: str = os.getcwd(),
  ):
    self.name = name
    self.directory = directory
    self.links = []

    save_path = os.path.join(self.directory, self.name, "Still Downloading")
    self.param = {"save_path": save_path}

  def add_links(self, links_str: str):
    link_lines = links_str.split('\n')

    for line in link_lines:
      uri_index = line.find('magnet:?')

      if uri_index == -1:
        continue

      magnet_link = line[uri_index:].split()[0]

      self.links.append(magnet_link)
      print(f"{self.name}:", 'Added Magnet Link:', magnet_link)


In [ ]:
@dataclass
class Download:
  bar: widgets.FloatSlider
  torrent: lt.torrent_handle

  # def __init__(self, layout, style, ses, magnet_link, params):
  #   self.bar = widgets.FloatSlider(
  #       step=0.01, disabled=True, layout=layout, style=style
  #   )
  #   torrent = lt.add_magnet_uri(ses,  magnet_link, params)
  #   ic(torrent)
  #   self.torrent = torrent



In [ ]:
class Downloader:

  downloads = []

  ses = lt.session({'allow_multiple_connections_per_ip': True,
    'dont_count_slow_torrents': True,
    'active_downloads': -1,
    'active_seeds': 7,
    'active_checking': -1,})

  layout = widgets.Layout(width="auto")
  style = {"description_width": "initial"}

  state_str = [
      "queued",
      "checking",
      "downloading metadata",
      "downloading",
      "finished",
      "seeding",
      "allocating",
      "checking fastresume",
  ]

  def __init__(self):
    self.ses.listen_on(6881, 6901)

  def add_category(self, category):
    params = category.param
    magnet_uris = category.links

    for magnet_uri in magnet_uris:
      # self.downloads.append(Download(self.layout, self.style, ses, magnet_uri, params))
      self.downloads.append(
        Download(
          widgets.FloatSlider(
            step=0.01, disabled=True, layout=self.layout, style=self.style
          ),
          lt.add_magnet_uri(self.ses,  magnet_uri, params),
        )
      )
    # ic(self.downloads)

  def _display_download_bars(self):
    download_bars = [
        download.bar
        for download in self.downloads
    ]
    # ic(download_bars)
    display(*download_bars)

    return download_bars

  def _close_torrent(self, download, download_bars: list):
    self.ses.remove_torrent(download.torrent)
    self.downloads.remove(download)
    download.bar.close()
    download_bars.remove(download.bar)

  def _move_downloaded_files(self, old_dir, new_dir):
    #TODO: add file moving mec
    os.rename(old_dir, new_dir)

  def start_downloading(self):
    moving_threads = []
    download_bars = self._display_download_bars()

    # ic(self.downloads)
    while self.downloads:

      for download in self.downloads:
        torrent = download.torrent
        bar = download.bar

        status = torrent.status()

        incomplete_download_dir = status.save_path
        complete_download_dir = os.path.dirname(incomplete_download_dir)

        incomplete_download_files_dir = os.path.join(
          incomplete_download_dir,
          torrent.name(),
        )
        complete_download_files_dir = os.path.join(
          complete_download_dir,
          torrent.name(),
        )

        # ic(download_bars)
        if os.path.exists(complete_download_files_dir): #s.state != 2 and
          print(torrent.name(), "Already Completed")
          self._close_torrent(download, download_bars)

          continue

        if download.torrent.is_seed():
          print("download completed")
          thread = threading.Thread(
            target=self._move_downloaded_files,
            args=(incomplete_download_files_dir, complete_download_files_dir),
          )
          thread.start()
          moving_threads.append(thread)

          # self._close_torrent(download, download_bars)

          continue

        download.bar.description = " ".join(
            [
                download.torrent.name(),
                str(status.download_rate / 1000),
                "kB/s",
                self.state_str[status.state],
            ]
        )
        download.bar.value = status.progress * 100

      time.sleep(1)

    for thread in moving_threads:
      thread.join()

# downloader = Downloader()
# downloader.add_category(shows_n_series)
# downloader.start_downloading()

In [ ]:
shows_n_series = Category(name="Shows", directory=os.getcwd())
shows_n_series.add_links("""
magnet:?xt=urn:btih:04F5E9F5DB4F07DE34FFDB1858A8144388AA32AA&dn=Andrew+Tate+-+Hustler+University&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce&tr=udp%3A%2F%2F47.ip-51-68-199.eu%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.internetwarriors.net%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2920%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce

""")

Shows: Added Magnet Link: magnet:?xt=urn:btih:04F5E9F5DB4F07DE34FFDB1858A8144388AA32AA&dn=Andrew+Tate+-+Hustler+University&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce&tr=udp%3A%2F%2F47.ip-51-68-199.eu%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.internetwarriors.net%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2920%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce


In [ ]:
downloader = Downloader()
downloader.add_category(shows_n_series)
downloader.start_downloading()

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

download completed
Andrew Tate - Hustler University Already Completed
